# Data exploration

Notebook looking at correlation between variables and correlation between variables and $C^*$

## Feature correlation

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
pad = Path(os.getcwd())
if pad.name == "ml_observation_operator":
    pad_correct = pad.parent
    os.chdir(pad_correct)

In [ ]:
ML_data_pad = Path("data/Zwalm_data/ML_data")
X_full_all = pd.read_pickle(ML_data_pad/"X_full_all.pkl")

y_train = pd.read_pickle(ML_data_pad/"y_train.pkl")
y_test = pd.read_pickle(ML_data_pad/"y_test.pkl")
y_full = pd.read_pickle(ML_data_pad/"y_full.pkl")

Cstar = pd.read_pickle(ML_data_pad/"Cstar.pkl")

X_train = pd.read_pickle(ML_data_pad/"X_train.pkl")
X_test = pd.read_pickle(ML_data_pad/"X_test.pkl")
X_full = pd.read_pickle(ML_data_pad/"X_full.pkl")
display(X_full.head())
print(X_full.shape)

X_train_small = pd.read_pickle(ML_data_pad/"X_train_small.pkl")
X_test_small = pd.read_pickle(ML_data_pad/"X_test_small.pkl")
X_full_small = pd.read_pickle(ML_data_pad/"X_full_small.pkl")
display(X_full_small.head())


In [ ]:
features_corr = X_full.corr(method = 'pearson')
features_corr.style.background_gradient(cmap = 'coolwarm')

In [ ]:
type(features_corr)
features_corr.columns

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(features_corr.values, cmap = 'coolwarm')
cb = fig.colorbar(im)
cb.set_label(r'$\rho$ [-]')
column_names = ['$\gamma^0_{\mathrm{VV,forest}}$','$\gamma^0_{\mathrm{VH,forest}}$','$\gamma^0_{\mathrm{VV,pasture}}$','$\gamma^0_{\mathrm{VH,pasture}}$','$\gamma^0_{\mathrm{VV,agriculture}}$','$\gamma^0_{\mathrm{VV,agriculture}}$','LAI$_{\mathrm{forest}}$','LAI$_{\mathrm{pasture}}$','LAI$_{\mathrm{agriculture}}$','Ascending',#'Descending',
                '$\Delta t$','DOY$_{\sin}$','DOY$_{\cos}$']
ax.set_xticks(range(len(features_corr.columns)), column_names, rotation=90)
ax.set_yticks(range(len(features_corr.columns)), column_names)

Now also the same plotted for the small dataset

In [ ]:
features_corr_small = X_full_small.corr(method = 'pearson')
features_corr_small.style.background_gradient(cmap = 'coolwarm')

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(features_corr_small.values, cmap = 'coolwarm')
cb = fig.colorbar(im)
cb.set_label(r'$\rho$ [-]')
column_names_small = ['$\gamma^0_{\mathrm{VV}}$','$\gamma^0_{\mathrm{VH}}$','LAI','Ascending',#'Descending',
                      '$\Delta t$','DOY$_{\sin}$','DOY$_{\cos}$']
ax.set_xticks(range(len(features_corr_small.columns)), column_names_small, rotation=90)
ax.set_yticks(range(len(features_corr_small.columns)), column_names_small)

Now combine the two figures

In [ ]:
fig, axes = plt.subplots(1,2, constrained_layout = True,figsize = (9,4.5))
im = axes[0].imshow(features_corr.values, cmap = 'coolwarm')
#cb = fig.colorbar(im)
axes[0].set_xticks(range(len(features_corr.columns)), column_names, rotation=90)
axes[0].set_yticks(range(len(features_corr.columns)), column_names)
axes[0].set_title('(a)')

im = axes[1].imshow(features_corr_small.values, cmap = 'coolwarm')
cb = fig.colorbar(im)
cb.set_label(r'$\rho$ [-]')
axes[1].set_xticks(range(len(features_corr_small.columns)), column_names_small, rotation=90)
axes[1].set_yticks(range(len(features_corr_small.columns)), column_names_small)
axes[1].set_title('(b)')

figures_pad = Path('Figures/Figures_chapter_ML_obs_op')
if not os.path.exists(figures_pad):
    os.makedirs(figures_pad)
fig.savefig(figures_pad/'feature_correlations.pdf',format = 'pdf')

Only as an experiment: PCA

In [ ]:
from statsmodels.multivariate.pca import PCA
pca_test = PCA(X_full, standardize=True)
display(pca_test.scores)
fig, ax = plt.subplots()
pca_test.plot_scree(ax = ax, ncomp = 10)
